In [5]:
!pip install keras
!pip install tensorflow

     ---------------------------------------- 1.1/1.1 MB 755.6 kB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 kB 7.7 MB/s eta 0:00:00
     -------------------------------------- 126.6/126.6 kB 7.8 MB/s eta 0:00:00
     -------------------------------------- 267.2/267.2 kB 8.0 MB/s eta 0:00:00


     -------------------                   199.5/385.0 MB 90.3 kB/s eta 0:34:15


ERROR: Exception:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 437, in _error_catcher
    yield
  File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 560, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 526, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\ProgramData\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\ProgramData\anaconda3\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "C:\ProgramData\anaconda3\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\ProgramData\anaconda3\lib\ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "C:\ProgramData\anaconda3\lib

In [6]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model, save_model

# Load custom preprocessing and feature selection functions
# from data import preprocess_data as custom_preprocess_data
# from feature_selection import tree_based_selection

def load_data(directory):
    """Load data from all CSV files in the specified directory."""
    df_main = pd.DataFrame()
    chunksize = 50000  # Adjust the chunk size as needed
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirname, filename)
            if file_path.endswith('.csv'):
                print(f"Loading file: {file_path}")
                for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
                    df_main = pd.concat([df_main, chunk], ignore_index=True)
    return df_main

def clean_data(df):
    """Clean the dataset by dropping duplicates, handling NaN values, and converting labels."""
    print("Columns before cleaning:", df.columns)
    print(df.shape)

    df.columns = df.columns.str.strip()
    print("Columns after stripping spaces:", df.columns)
    print(df.shape)

    if 'Label' in df.columns:
        df['Label'] = df['Label'].apply(lambda x: 0 if isinstance(x, str) and x.lower().startswith("benign") else 1)
    else:
        raise KeyError("The 'Label' column is missing from the dataset.")

    print(df.isna().sum())
    df.drop_duplicates(inplace=True)
    print("After dropping duplicates:", df.shape)
    print(df.shape)

    # df.dropna(inplace=True)
    # print("After handling NaN values:", df.shape)
    # print(df.shape)
    return df

def custom_preprocess_data(df):
    """Preprocess the dataset by converting non-numeric data, normalizing numeric data, and splitting."""
    print("Initial columns:", df.columns)

    df.columns = df.columns.str.strip()

    if 'Timestamp' in df.columns:
        print("Dropping Timestamp column...")
        df.drop(['Timestamp'], axis=1, inplace=True)
    else:
        print("Timestamp column not found. Proceeding without dropping.")

    print("Converting object types to numeric...")
    df_numeric = convert_to_numeric(df.copy())
    print("Columns after conversion to numeric:", df_numeric.columns)

    if 'Protocol' in df_numeric.columns:
        print("One-hot encoding Protocol column...")
        df_numeric = pd.get_dummies(df_numeric, columns=['Protocol'])
        print("Columns after one-hot encoding:", df_numeric.columns)
    else:
        print("Protocol column not found. Skipping one-hot encoding.")

    protocol_cols = ["Protocol_0", "Protocol_17", "Protocol_6"]
    print("Converting Protocol columns to int64...")
    for col in protocol_cols:
        if col in df_numeric.columns:
            df_numeric[col] = df_numeric[col].astype('int64')

    if 'Dst Port' in df_numeric.columns:
        print("Converting Dst Port column to numeric...")
        df_numeric['Dst Port'] = pd.to_numeric(df_numeric['Dst Port'], errors='coerce')
        print("Handling NaNs in Dst Port column...")
        df_numeric['Dst Port'].fillna(0, inplace=True)
        df_numeric['Dst Port'] = df_numeric['Dst Port'].astype('int64')

    print("Handling infinite and very large values...")
    df_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_numeric.fillna(df_numeric.mean(), inplace=True)

    print("Normalizing numeric columns...")
    numeric_cols = df_numeric.columns.difference(['Label'])

    print("Numeric columns being normalized:", numeric_cols)
    print("Shape of numeric data before normalization:", df_numeric[numeric_cols].shape)

    scaler = MinMaxScaler()
    df_numeric[numeric_cols] = scaler.fit_transform(df_numeric[numeric_cols])
    joblib.dump(scaler, 'scaler.pkl')

    if 'Label' in df_numeric.columns:
        df_numeric.insert(len(df_numeric.columns) - 1, 'Label', df_numeric.pop('Label'))

    print("Preprocessing complete. Final columns:", df_numeric.columns)
    return df_numeric

def tree_based_selection(df, target_col='Label', num_features=20):
    """
    Perform feature selection using a tree-based method (e.g., Random Forest).
    Args:
        df (pd.DataFrame): The dataset including features and target.
        target_col (str): The name of the target column.
        num_features (int): The number of top features to select.
    Returns:
        pd.DataFrame: Dataset with selected features.
    """
    # Split the data into features and target
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Initialize the Random Forest model
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Fit the model
    rf.fit(X, y)

    # Get feature importances
    importances = rf.feature_importances_

    # Create a DataFrame for feature importances
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

    # Sort the features by importance
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Select the top features
    top_features = feature_importances['Feature'].head(num_features).tolist()

    # Return the dataset with only the top features and the target column
    return df[top_features + [target_col]]


def convert_to_numeric(df):
    """Convert object-type features (except 'Label') to numeric."""
    numeric_cols = df.columns.difference(['Dst Port', 'Protocol', 'Label'])
    for col in numeric_cols:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            pass
    return df

def build_ann_model(input_dim):
    """Build and compile an ANN model using Keras."""
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


existing_model_path = "C:/Users/Admin/Documents/Final/HIDSProject/models/cicids2017_random_forest_model.pkl"
new_data_directory = "C:/Users/Admin/Documents/Final/HIDSProject/data/Trainers"
updated_model_path = "C:/Users/Admin/Documents/Final/HIDSProject/models/"





ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print("Building a new ANN model...")
ann_model = build_ann_model(input_dim=22)

In [ ]:
# Load and preprocess the new data
print("Loading new data...")
df_new = load_data(new_data_directory)
print("New data loaded. Shape:", df_new.shape)